# Installing Libraries

In [ ]:
!pip install -q scikit-learn numpy pandas altair

# Performance Measures
이번엔 성능 측정치를 탐색해보자. 그 전에, 먼저 생각해봐야 할 게 있다. 과연 어느 정도 성능 측정치가 나와야 **나쁘지는 않다**고 할 수 있을까? 즉, 성능치의 **하한**을 어떻게 정하면 좋을까? 이를 위해서 참조용으로 사용할 기계 학습 모델Reference Model을 만들어보자.

scikit-learn에서는 이를 위해서 Dummy 모델이라는 것을 지원하고 있다. 분류 과업의 경우 [sklearn.dummy.DummyClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html#sklearn.dummy.DummyClassifier), 회귀 과업의 경우 [sklearn.dummy.DummyRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html#sklearn.dummy.DummyRegressor)로 활용 가능하다.

Dummy 모델이 하는 일은 간단하다. 일반 기계 학습 모델이 하는 것처럼 특성값에 대응하는 매개변수를 학습하는 게 아니라, 특성값과 상관없이 독립적으로 레이블 값의 분포만을 활용해서 결과값을 내는 것이다.

예를 들어, 분류 과업에서 훈련 데이터셋의 레이블 분포가 0: 30 vs. 1: 70이라고 가정하자. 그럼 범주형 레이블 값은 **1**이 더 많으므로 항상 1을 출력한다. 각 레이블에 대한 확률값은 훈련 데이터셋의 분포를 참고하여, 레이블 값 **0**에 대해서는 0.3, **1**에 대해서는 0.7을 출력하는 것이다. 단순히 생각해봐도 특성값과 레이블값 간의 관계를 파악하려는 우리의 학습 모델이, 이런 Dummy 모델보다 못하다는 건 말도 되지 않는다. 따라서, 이런 Dummy 모델이 내는 성능치를 하한값으로 삼는 것이다.




## Threshold Metric
먼저, 학습 모델의 출력값이 범주형일 때 사용하는 Threshold Metric을 사용해보자. 다음과 같은 성능 측정치를 비교해보겠다:
* Confusion Matrix: [sklearn.metrics.confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix)
* Accuracy: [sklearn.metrics.accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score)
* Precision: [sklearn.metrics.precision_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score)
* Recall: [sklearn.metrics.recall_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score)
* F1 Score: [sklearn.metrics.f1_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score)
* Balanced Accuracy: [sklearn.metrics.balanced_accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html#sklearn.metrics.balanced_accuracy_score)
* G-Mean: 별도 구현체 없음
* Matthew's Correlation Coefficient: [sklearn.metrics.matthews_corrcoef](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html#sklearn.metrics.matthews_corrcoef)

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


X, y = make_classification(
    n_samples=1000, # 총 샘플 개수
    n_features=15, # 특성값 수
    n_redundant=1, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.9, 0.1], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

splitter = StratifiedKFold(
    n_splits=5, # 생성할 Split의 갯수
    shuffle=True, # 전체 샘플의 40%를 검증 데이터로 활용
    random_state=42
)

scores = []

for I_train, I_test in splitter.split(X, y):
    X_train, X_test, y_train, y_test = X[I_train, :], X[I_test], y[I_train], y[I_test]

    model_dummy = DummyClassifier(
        strategy='prior' # 위에서 말한 예처럼, 범주형 레이블 값은 Majority Class, 확률값은 훈련 데이터셋의 분포로부터 나오는 설정이다.
    )
    model_lr = LogisticRegression()

    for name, model in zip(['Dummy', 'LR'], [model_dummy, model_lr]):
        y_pred = model.fit(X=X_train, y=y_train).predict(X_test)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        score = {
            'Model': name,
            'TN': tn,
            'FP': fp,
            'FN': fn,
            'TP': tp,
            'Accuracy': accuracy_score(y_test, y_pred),
            'Precision (Pos. = 0)': precision_score(
                y_test, y_pred, pos_label=0 # Positive Class를 무엇으로 설정할 것인지를 의미한다.
            ),
            'Precision (Pos. = 1)': precision_score(y_test, y_pred, pos_label=1),
            'Recall (Pos. = 0)': recall_score(y_test, y_pred, pos_label=0),
            'Recall (Pos. = 1)': recall_score(y_test, y_pred, pos_label=1),
            'F1 (Pos. = 0)': f1_score(y_test, y_pred, pos_label=0),
            'F1 (Pos. = 1)': f1_score(y_test, y_pred, pos_label=1),
            'Bal. Acc': balanced_accuracy_score(y_test, y_pred),
            'G-Mean': np.sqrt(recall_score(y_test, y_pred, pos_label=0) * recall_score(y_test, y_pred, pos_label=1)), # G-Mean은 직접 구현했다.
            'MCC': matthews_corrcoef(y_test, y_pred)
        }
        scores.append(score)

scores = pd.DataFrame(scores)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defin

In [ ]:
scores.groupby('Model').mean()

,TN,FP,FN,TP,Accuracy,Precision (Pos. = 0),Precision (Pos. = 1),Recall (Pos. = 0),Recall (Pos. = 1),F1 (Pos. = 0),F1 (Pos. = 1),Bal. Acc,G-Mean,MCC
Model,,,,,,,,,,,,,,
Dummy,180.0,0.0,20.0,0.0,0.900,0.900000,0.0000,1.000000,0.00,0.947368,0.000000,0.500000,0.000000,0.000000
LR,179.8,0.2,4.4,15.6,0.977,0.976231,0.9875,0.998889,0.78,0.987400,0.867896,0.889444,0.880774,0.864835


위의 결과를 해석해보자.
* Accuracy: 0.9 vs. 0.97
  * 0.07은 기계 학습에서 꽤나 큰 차이라고 할 수 있다. 하지만, 비교 대상이 Dummy 모델이라면? 레이블 분포만을 참고해서 이 정도의 성능을 내었다는 게 꽤나 놀랍다.
  이런 문제 때문에, 레이블 분포가 불균형한 문제에 대해서는 Accuracy를 쓰지 않는 것이다.
* Precision (Pos. = 0): 0.9 vs. 0.97
  * 실제 Positive Class ( = Minority Class)는 0이 아닌 1이지만, 만약 이를 잘못 선정하면 어떻게 되는지를 보여주기 위해 이 수치를 계산했다.
  * 보다시피, Dummy 모델도 굉장히 높은 성능치를 내는 것을 알 수 있다. Positive Class를 잘못 선택하면 이처럼 잘못된 해석을 낼 수 있다.
* Precision (Pos. = 1): 0.0 vs. 0.99
  * 정상적으로 Minority Class를 선정했을때는 Dummy 모델의 성능 측정치가 0이 나오게 된다. Confusion Matrix의 내용을 보면 Dummy 모델의 경우, Precision 계산시에 활용되는 FP 및 TP 모두 0이며, Precision = $\frac{TP}{TP + FP}$이므로, 0으로 나누는 문제가 발생한다 (위의 오류가 그에 대한 내용이다). 이 때는 (기본적으로) Precision 값을 0으로 바꾸게 된다.
* Recall과 F1
  * Precision 처럼 Positive Class를 어떻게 선택하느냐에 따라 성능이 바뀌는 측정치는 주의 깊게 Positive Class를 선택해야 한다.
* Balanced Accuracy: 0.5 vs. 0.89
  * Minority Class에 대한 Recall, Majority Class에 대한 Recall 값의 산술 평균이다.
  *  Dummy 모델의 경우 0.5가 나왔는데, 각 Class을 예측하는 데 반 정도는 맞춘다고 해석이 될 수 있다.
* G-Mean: 0.0 vs. 0.89
  * Minority Class에 대한 Recall, Majority Class에 대한 Recall 값의 기하 평균이다.
  * Balanced Accuracy와는 달리 Dummy 모델의 경우 아예 0가 나왔다.
  * G-Mean은 각 레이블 값을 어느 정도 균형 있게 예측할 수 있어야 높은 값이 나올 수 있다.
* Matthew's Correlation Coefficient: 0.0 vs. 0.86
  * Dummy 모델의 경우 0이 나왔다. 이 말인 즉슨, Dummy 모델은 무작위 모델과 같은 성능을 나타낸다는 의미다.
  * TP, FP, TN, FN 모두를 고려하는 MCC는 특히 레이블 불균형이 심한 경우에 다른 측정치보다 나은 대안이 될 수 있다.


## Probability Metric
이번엔 다음과 같은 확률값을 기반으로 한 성능 측정치를 써보자:
* Logistic Loss: [sklearn.metrics.log_loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html#sklearn.metrics.log_loss)
* Brier Loss: [sklearn.metrics.brier_score_loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.brier_score_loss.html#sklearn.metrics.brier_score_loss)
* Brier Skill Score: 별도 구현체 없음

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import brier_score_loss, log_loss


X, y = make_classification(
    n_samples=1000, # 총 샘플 개수
    n_features=15, # 특성값 수
    n_redundant=1, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.9, 0.1], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

splitter = StratifiedKFold(
    n_splits=5, # 생성할 Split의 갯수
    shuffle=True, # 전체 샘플의 40%를 검증 데이터로 활용
    random_state=42
)

scores = []

for I_train, I_test in splitter.split(X, y):
    X_train, X_test, y_train, y_test = X[I_train, :], X[I_test], y[I_train], y[I_test]

    model_dummy = DummyClassifier(
        strategy='prior' # 위에서 말한 예처럼, 범주형 레이블 값은 Majority Class, 확률값은 훈련 데이터셋의 분포로부터 나오는 설정이다.
    )
    model_lr = LogisticRegression()
    model_rf = RandomForestClassifier()
    score = dict()
    for name, model in zip(['Dummy', 'LR', 'RF'], [model_dummy, model_lr, model_rf]):
        y_pred = model.fit(X=X_train, y=y_train).predict_proba(X_test) # 확률값을 출력해야 함을 잊지 말자.
        score[f'{name} - Log loss'] = log_loss(y_test, y_pred)
        score[f'{name} - Brier loss'] = brier_score_loss(y_test, y_pred[:, 1], pos_label=1)
    scores.append(score)

scores = pd.DataFrame(scores).assign(
    Brier_Skill_LR = lambda x: 1 - x['LR - Brier loss'] / x['Dummy - Brier loss'],
    Brier_Skill_RF = lambda x: 1 - x['RF - Brier loss'] / x['Dummy - Brier loss']
)

In [ ]:
scores.mean()

Dummy - Log loss      0.325083
Dummy - Brier loss    0.090000
LR - Log loss         0.092445
LR - Brier loss       0.019031
RF - Log loss         0.159582
RF - Brier loss       0.022940
Brier_Skill_LR        0.788542
Brier_Skill_RF        0.745112
dtype: float64

* Log loss: 0.325 vs. 0.092 vs. 0.160
    * Logistic Regression이 가장 좋고, 그 다음이 Random Forest, Dummy 순이다.
* Brier loss: 0.09 vs. 0.019 vs. 0.023
    * Log loss와 같은 결과다. 하지만, Log loss와는 달리 Logistic Regression 과 Random Forest 간의 차이가 작은 것을 알 수 있다.
* Brier skill score: 0.78 vs. 0.75
    * 대신, Brier skill score를 활용하면 보다 명확하게 두 모델 간의 성능 차이를 확인할 수 있다.



## Ranking Metric
마지막으로 Ranking Metric을 사용해보자. 먼저, 하나의 정량화된 수치를 사용하기 전에, 여러 임계치에 대한 성능의 경향성을 보여주는 Recieving Operation Characteristic (ROC)([sklearn.metrics.roc_curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html#sklearn.metrics.roc_curve))과 Precision-Recall Curve (Precision-Recall Curve: [sklearn.metrics.precision_recall_curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html#sklearn.metrics.precision_recall_curve))부터 확인해보자.

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_curve, precision_recall_curve


X, y = make_classification(
    n_samples=1000, # 총 샘플 개수
    n_features=15, # 특성값 수
    n_redundant=1, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.9, 0.1], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

splitter = StratifiedShuffleSplit(
    test_size=0.4,
    n_splits=1,
    random_state=42
)

I_train, I_test = next(splitter.split(X, y))
X_train, X_test, y_train, y_test = X[I_train, :], X[I_test], y[I_train], y[I_test]

model_dummy = DummyClassifier(
    strategy='prior' # 위에서 말한 예처럼, 범주형 레이블 값은 Majority Class, 확률값은 훈련 데이터셋의 분포로부터 나오는 설정이다.
)
model_lr = LogisticRegression()
model_rf = RandomForestClassifier()

rocs, prcs = [], []

for name, model in zip(['Dummy', 'LR', 'RF'], [model_dummy, model_lr, model_rf]):
    y_pred = model.fit(X=X_train, y=y_train).predict_proba(X_test) # 확률값을 출력해야 함을 잊지 말자.
    fpr, tpr, thresholds = roc_curve(y_test, y_pred[:, 1], pos_label=1)
    roc = pd.DataFrame({
        'model': name, 'fpr': fpr, 'tpr': tpr, 'thresholds': thresholds
    })
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred[:, 1], pos_label=1)
    prc = pd.DataFrame({
        'model': name, 'precision': precision, 'recall': recall, 'thresholds': np.concatenate([thresholds, [-1]])
    })

    rocs.append(roc)
    prcs.append(prc)

rocs = pd.concat(rocs, axis=0, ignore_index=True)
prcs = pd.concat(prcs, axis=0, ignore_index=True)

In [ ]:
import altair as alt


chart_roc = alt.Chart(rocs).mark_line().encode(
    x='fpr:Q', y='tpr:Q', color='model:N'
).properties(
    title='Receiving Operating Charateristic'
)

chart_prc = alt.Chart(prcs).mark_line().encode(
    x='recall:Q', y='precision:Q', color='model:N'
).properties(
    title='Precision-Recall Curve'
)

chart_roc | chart_prc

alt.HConcatChart(...)

먼저 ROC를 보자
* Dummy의 경우 FPR (Type-1 Error)이 증가함에 따라 TPR (Recall)도 같이 선형적으로 증가함을 알 수 있다. 즉, 임계치 변화에 따라 오류율과 성능이 같이 증가함을 의미한다.
* Logistic Regression 과 Random Forest의 경우 FPR이 약 0.1일 때, TPR이 약 0.9를 기록한다. 그 이후부터는 FPR은 크게 증가하는 반면(오류율이 크게 증가) TPR은 낮게 증가한다(Positive Class를 구분하는 능력은 크게 증가하지 않음). 즉, FPR이 약 0.1, TPR이 약 0.9를 기록하는 지점의 임계치가 괜찮은 임계치라고 생각할 수 있다.

다음은 PRC를 보자.
* Dummy의 경우 Recall이 증가할수록 Precision은 거의 선형적으로 감소하는 것을 알 수 있다. 즉, 임계치 변화에 따라서 전체 Positive Class 중에 실제 Positive Class를 구분하는 능력은 증가하는 반면 (Recall), Negative Class 또한 Positive Class로 구분하는 비율 (Precision)이 증가한다고 생각할 수 있다.
* Logistic Regression 과 Random Forest의 경우 Recall이 약 0.7 ~ 0.8일때까지는 Precision이 0.9 이상으로 유지되고 있다. 즉, Negative Class를 Positive Class로 예측하는 경우는 크게 없으면서(Precision), 실제 Positive Class를 Positive Class로 잘 구분함을 의미한다. Recall이 0.8 이후부터는 급격하게 Precision이 감소하며, 이는 많은 수의 실제 Negative Class들을 Positive Class로 예측하고 있다는 의미이다.

이러한 점을 참고하면 7주차에 했던 Threshold Tuning을 쉽게 할 수 있다. 각 임계치 별로 주어진 성능 측정치를 최대화 할 수 있는 지점이 바로 최적의 임계치인 것이다. 일일이 임계치별로 성능치를 내는 것보다, 이미 ROC에 활용된 TPR과 FPR을 활용해보자.

먼저, FPR은 사실 간단하게 다음과 같이 Specificity (Negative Class를 잘 구분하는 비율)로 변환할 수 있다.
\begin{align}
\text{FPR} &= \frac{FP}{FP + TN}\\
\text{Specficity} &= \frac{TN}{FP + TN}\\
&= 1 - \text{FPR}
\end{align}
이로부터, TPR (Recall)과 Sensitivity를 활용한 측정치인 Balanced Accuracy 또는 G-Mean을 고려해볼 수 있다. 실습에서는 G-Mean을 해보자.


In [ ]:
import numpy as np


rocs.assign(
    g_mean = lambda x: np.sqrt( (1 - x['fpr']) * x['tpr'] )
).iloc[
    lambda x: x.groupby('model')['g_mean'].idxmax(), :
]

,model,fpr,tpr,thresholds,g_mean
0,Dummy,0.000000,0.000,1.100000,0.000000
14,LR,0.072222,0.925,0.055137,0.926388
40,RF,0.052778,0.925,0.180000,0.936045


즉, Logistic Regression은 임계치를 0.055, Random Forest는 0.180으로 잡을 때 G-Mean 값을 최대화 할 수 있다는 의미다. 실제로 그럴까?

In [ ]:
import numpy as np
from sklearn.metrics import recall_score


print('Logistic Regression')
for threshold in [0.055, 0.1, 0.3, 0.5, 0.8]:
    y_pred = np.where(model_lr.predict_proba(X_test)[:, 1] >= threshold, 1, 0)
    score = np.sqrt(recall_score(y_test, y_pred, pos_label=0) * recall_score(y_test, y_pred, pos_label=1))
    print(f' - Threshold = {threshold} / G-Mean = {score: .3f}')

print('\nRandom Forest Regression')
for threshold in [0.1, 0.180, 0.3, 0.5, 0.8]:
    y_pred = np.where(model_rf.predict_proba(X_test)[:, 1] >= threshold, 1, 0)
    score = np.sqrt(recall_score(y_test, y_pred, pos_label=0) * recall_score(y_test, y_pred, pos_label=1))
    print(f' - Threshold = {threshold} / G-Mean = {score: .3f}')

Logistic Regression
 - Threshold = 0.055 / G-Mean =  0.926
 - Threshold = 0.1 / G-Mean =  0.918
 - Threshold = 0.3 / G-Mean =  0.892
 - Threshold = 0.5 / G-Mean =  0.880
 - Threshold = 0.8 / G-Mean =  0.837

Random Forest Regression
 - Threshold = 0.1 / G-Mean =  0.914
 - Threshold = 0.18 / G-Mean =  0.936
 - Threshold = 0.3 / G-Mean =  0.889
 - Threshold = 0.5 / G-Mean =  0.835
 - Threshold = 0.8 / G-Mean =  0.758


놀랍게도 그러하다! 이런 식으로 최적의 Threshold를 찾을 수 있다.

Precision-Recall Curve는 어떨까? 가장 간단하게 계산할 수 있는 건 F1 Score 다.

In [ ]:
import numpy as np


prcs.assign(
    f1 = lambda x:  (2 * x['precision'] * x['recall']) / (x['precision'] + x['recall'])
).iloc[
    lambda x: x.groupby('model')['f1'].idxmax(), :
]

,model,precision,recall,thresholds,f1
0,Dummy,0.100000,1.000,0.100000,0.181818
369,LR,0.969697,0.800,0.475571,0.876712
426,RF,0.891892,0.825,0.270000,0.857143


즉, Logistic Regression은 임계치를 0.475, Random Forest는 0.270으로 잡을 때 F1 Score를 최대화 할 수 있다는 의미다. 한번 확인해보자.

In [ ]:
import numpy as np
from sklearn.metrics import f1_score


print('Logistic Regression')
for threshold in [0.1, 0.3, 0.475, 0.5, 0.8]:
    y_pred = np.where(model_lr.predict_proba(X_test)[:, 1] >= threshold, 1, 0)
    score = f1_score(y_test, y_pred)
    print(f' - Threshold = {threshold} / G-Mean = {score: .3f}')

print('\nRandom Forest Regression')
for threshold in [0.1, 0.270, 0.3, 0.5, 0.8]:
    y_pred = np.where(model_rf.predict_proba(X_test)[:, 1] >= threshold, 1, 0)
    score = f1_score(y_test, y_pred)
    print(f' - Threshold = {threshold} / G-Mean = {score: .3f}')

Logistic Regression
 - Threshold = 0.1 / G-Mean =  0.795
 - Threshold = 0.3 / G-Mean =  0.865
 - Threshold = 0.475 / G-Mean =  0.877
 - Threshold = 0.5 / G-Mean =  0.873
 - Threshold = 0.8 / G-Mean =  0.824

Random Forest Regression
 - Threshold = 0.1 / G-Mean =  0.661
 - Threshold = 0.27 / G-Mean =  0.857
 - Threshold = 0.3 / G-Mean =  0.842
 - Threshold = 0.5 / G-Mean =  0.812
 - Threshold = 0.8 / G-Mean =  0.730


보다시피, 위에서 확인한 임계치대로 설정하는 것이 최적의 F1 Score를 냄을 알 수 있다.

물론, ROC와 PRC는 어디까지나 서로 다른 임계치 값에 대한 경향성을 나타내는 것이지 하나의 정량적 수치는 아니다. 이런 경향성을 요약하는 대표적인 방법은 그래프 아래의 구역의 너비를 구하는 Area Under Curve (AUC)다. 각각에 대해서는 다음과 같은 함수를 활용하면 된다.
* ROC-AUC: [sklearn.metrics.roc_auc_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score)
* PR Curve-AUC: [sklearn.metrics.average_precision_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score)

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score, average_precision_score


X, y = make_classification(
    n_samples=1000, # 총 샘플 개수
    n_features=15, # 특성값 수
    n_redundant=1, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.9, 0.1], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

splitter = StratifiedKFold(
    n_splits=5, # 생성할 Split의 갯수
    shuffle=True, # 전체 샘플의 40%를 검증 데이터로 활용
    random_state=42
)

scores = []

for I_train, I_test in splitter.split(X, y):
    X_train, X_test, y_train, y_test = X[I_train, :], X[I_test], y[I_train], y[I_test]

    model_dummy = DummyClassifier(
        strategy='prior' # 위에서 말한 예처럼, 범주형 레이블 값은 Majority Class, 확률값은 훈련 데이터셋의 분포로부터 나오는 설정이다.
    )
    model_lr = LogisticRegression()
    model_rf = RandomForestClassifier()
    score = dict()
    for name, model in zip(['Dummy', 'LR', 'RF'], [model_dummy, model_lr, model_rf]):
        y_pred = model.fit(X=X_train, y=y_train).predict_proba(X_test) # 확률값을 출력해야 함을 잊지 말자.
        score[f'{name} - ROC-AUC'] = roc_auc_score(y_test, y_pred[:, 1])
        score[f'{name} - PRC-AUC'] = average_precision_score(y_test, y_pred[:, 1])
    scores.append(score)

pd.DataFrame(scores).mean()

Dummy - ROC-AUC    0.500000
Dummy - PRC-AUC    0.100000
LR - ROC-AUC       0.966389
LR - PRC-AUC       0.928299
RF - ROC-AUC       0.964639
RF - PRC-AUC       0.912895
dtype: float64